In [1]:
import math
import random
import time
from graph.graph import read_graph_from_csv

In [2]:
from stable_matching.TaskWorker import outward_satisfactory,overall_satisfactory,gamma_workers,individual_rationality_tasks,waste_pairwise
def estimate(Tasks, Workers):
    result_dict = {'fairness-pairwise': outward_satisfactory(Tasks, Workers),
                   # 'overall_satisfactory': overall_satisfactory(tasks,workers),
                   'individual_rationality': individual_rationality_tasks(Tasks),
                   'waste-pairwise': waste_pairwise(Tasks, Workers)}
    Sum = 0
    Max = 0
    Min = 1000
    for t_ in Tasks:
        if len(t_.students()) <=0:
            break
        q = t_.Q * gamma_workers(t_.R, t_.students()) / len(t_.R) / sum([t_.costs[e] for e in t_.students()])
        Sum += q
        if q> Max:
            Max = q
        if q< Min:
            Min = q
    result_dict['average-density'] = Sum/len(Tasks)
    result_dict['maximum-density'] = Max
    result_dict['minimum-density'] = Min
    return result_dict

In [3]:
graph_file = '../graphs/dash/dash.csv'

In [4]:
m = 10 # number of tasks
n = 100 # number of candidate workers

initialize workers by their ids

In [5]:
from stable_matching.TaskWorker import Worker

g = read_graph_from_csv(graph_file,0)
workers = []
worker_ids = random.sample(list(g.nodes.keys()),n)
for i in range(n):
    workers.append(Worker(idx = worker_ids[i]))
del g

initialize tasks : (ids, budget, reverse reachable set, Q)

In [6]:
from graph.QIM import sampling
from utils.funcs import max_k
from stable_matching.TaskWorker import Task
import scipy.stats as stats

mu = 1.0
sigma = 0.1

tasks = []
graph_ids = random.sample(range(100),m)

# allocate budget to every task, sum of all budget is total_budget
avg_budget  = 10
max_variance = math.ceil(avg_budget/2)


budgets = [random.randint(avg_budget-max_variance,avg_budget+max_variance) for _ in range(m)]

for i in range(m):
    G = read_graph_from_csv(graph_file,graph_ids[i])
    
    # generate cost of all candidate worker, task cost is generate from a trunc gaussian distribution mu = 1.0, sigma = 0.1, lb = 0.5, ub = 1.5
    X = stats.truncnorm((0.5-mu)/sigma, (1.5-mu)/sigma, loc= mu, scale= sigma).rvs(n)
    costs = {}
    for j in range(n):
        costs[workers[j]] = X[j]
        
    # values of all workers
    values = {}
    Q  = 0.0
    for v in G.nodes:
        values[v] = random.uniform(0.0, 1.0)
        Q += values[v]
        
    budget = budgets[i]
    print('budget:{}'.format(budget))
    start = time.time()
    # generate hyper graph of reverse reachable set in graph G
    k = max_k(budget, costs)
    print('k:{}'.format(k))
    RR = sampling(graph=G, C=worker_ids, k=k, delta=1/n, epsilon=0.01, values=values)
    print('time to generate RR:{}'.format(time.time()-start))
    
    #initialize tasks
    tasks.append(Task(idx=i, budget=budget, R=RR, Q=Q))
    tasks[i].initialize(costs)

budget:5
k:6
i_max:35
time to generate RR:0.6486344337463379
budget:13
k:15
i_max:34
time to generate RR:0.8114962577819824
budget:9
k:10
i_max:35
time to generate RR:0.7525885105133057
budget:9
k:11
i_max:35
time to generate RR:0.7686672210693359
budget:5
k:6
i_max:36
time to generate RR:2.553366184234619
budget:6
k:8
i_max:35
time to generate RR:0.760444164276123
budget:7
k:8
i_max:35
time to generate RR:1.4485135078430176
budget:14
k:16
i_max:34
time to generate RR:0.9404504299163818
budget:10
k:12
i_max:35
time to generate RR:1.6675832271575928
budget:15
k:17
i_max:34
time to generate RR:1.0056607723236084


In [7]:
for worker in workers:
    value_dict = {}
    for t in tasks:
        value_dict[t] = random.random()
    worker.set_preference(value_dict)

In [8]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_max_cover()
generalized_da(tasks,workers)
estimate(tasks,workers)

{'fairness-pairwise': 0.01851851851851849,
 'individual_rationality': [[5, 4.977195049786301],
  [13, 5.819660491271586],
  [9, 2.9483119456842006],
  [9, 5.003516720451021],
  [5, 4.991332742215252],
  [6, 3.892358188681466],
  [7, 4.918852924105278],
  [14, 7.43571387019767],
  [10, 4.79437014676433],
  [15, 9.111563874040943]],
 'waste-pairwise': 7.481481481481482,
 'average-density': 3.104812383542731,
 'maximum-density': 8.532429089316741,
 'minimum-density': 1.1949637337281294}

In [9]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_budget()
generalized_da(tasks,workers)
estimate(tasks, workers)

{'fairness-pairwise': 0.9666666666666667,
 'individual_rationality': [[5, 4.533971953504047],
  [13, 12.997260473248437],
  [9, 8.79746728688131],
  [9, 8.192857217495563],
  [5, 4.1591289383493955],
  [6, 5.9364384842691535],
  [7, 6.692214405488827],
  [14, 13.25103425787727],
  [10, 9.711031858661915],
  [15, 14.886366792896794]],
 'waste-pairwise': 0.011111111111111112,
 'average-density': 1.9910721442551491,
 'maximum-density': 3.629351664738633,
 'minimum-density': 0.8428212179308112}

In [10]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_matroid(4)
generalized_da(tasks,workers)
estimate(tasks,workers)

{'fairness-pairwise': -1.4047619047619047,
 'individual_rationality': [[5, 2.530037898242301],
  [13, 4.851378678005616],
  [9, 4.005435059100366],
  [9, 4.711975196078452],
  [5, 2.8495759493286457],
  [6, 3.0597420430521654],
  [7, 4.008110402884159],
  [14, 5.069755708404464],
  [10, 4.6513561797836145],
  [15, 5.291359904101123]],
 'waste-pairwise': 15.214285714285714,
 'average-density': 3.535422951148506,
 'maximum-density': 7.488326480578932,
 'minimum-density': 2.0485712066873973}

In [11]:
# from stable_matching.stableMatching import heuristic
# for worker in workers:
#     worker.refresh()
# for task in tasks:
#     task.refresh()
# heuristic(tasks,workers,100)
# estimate(tasks,workers)